<a href="https://colab.research.google.com/github/dutt2022/Coursera-Data-science-Projects/blob/main/rf_forest_9_targets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'financial-performance-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F70781%2F7733789%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240307%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240307T124306Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8b61a914e9f2d145fb042e797e08728dd1f2b14642473656b188f1586738c71eddb4403104a5743d570a5f5d04ff7bd4c29f8c97088d6d9253913001eadfdf4654d24d831094cae567e27f74aa76b7cb52f12e6fdd9c0fd650d32baf5a4bcf5dc45d02aabccfeff8cc150b3351b2c2916c1f6777296fc60488e53ca661be590e77bb6ba5eeb953d0882b5d3967c3a6ea1388c4e57414cdea9f60d5975875b5558855e9c312cb1c4cf7749f45700b894ef1f9c96240af00ce6905f125e3df58b00aea78f4fe217a43a7f4f275871c1c309107401a74f2ec32d770768dca6c2812d581ff1d92b2b83155fb76eb6caf63085bfb245a8a31d01edbd0fcc030948857'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1436515 bytes downloaded
Downloaded and uncompressed: financial-performance-prediction
Data source import complete.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import cross_val_score as cvs
from xgboost import XGBRegressor

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/financial-performance-prediction/data_dictionary.txt
/kaggle/input/financial-performance-prediction/sample_submission.csv
/kaggle/input/financial-performance-prediction/test.csv
/kaggle/input/financial-performance-prediction/train.csv


In [3]:
train = pd.read_csv('/kaggle/input/financial-performance-prediction/train.csv')
test = pd.read_csv('/kaggle/input/financial-performance-prediction/test.csv')

## Look at data shape, dtypes and unite train & test into a single DF

In [4]:
print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')

Train shape: (1624, 212)
Test shape: (406, 203)


In [5]:
targets_names = [col for col in train if col not in test]
print(f'Target columns: {targets_names}')

Target columns: ['Q0_TOTAL_ASSETS', 'Q0_TOTAL_LIABILITIES', 'Q0_TOTAL_STOCKHOLDERS_EQUITY', 'Q0_GROSS_PROFIT', 'Q0_COST_OF_REVENUES', 'Q0_REVENUES', 'Q0_OPERATING_INCOME', 'Q0_OPERATING_EXPENSES', 'Q0_EBITDA']


In [6]:
train.dtypes.value_counts()

float64    207
object       4
int64        1
dtype: int64

In [7]:
test.dtypes.value_counts()

float64    198
object       4
int64        1
dtype: int64

In [8]:
# make an easy col to split back the train and test sets
train['is_train'] = True
test['is_train'] = False
df = pd.concat([train, test], axis = 0, ignore_index = True)

## Deal with string/categoric columns

In [9]:
df.select_dtypes(include = 'O')

,industry,sector,recommendationKey,financialCurrency
0,Personal Services,Consumer Cyclical,strong_buy,USD
1,Building Products & Equipment,Industrials,buy,USD
2,NaN,Unknown,NaN,NaN
3,Scientific & Technical Instruments,Technology,hold,USD
4,Drug Manufacturers - Specialty & Generic,Healthcare,buy,USD
...,...,...,...,...
2025,Packaged Foods,Consumer Defensive,hold,USD
2026,NaN,Unknown,NaN,NaN
2027,Medical Devices,Healthcare,buy,USD
2028,Drug Manufacturers - Specialty & Generic,Healthcare,buy,USD


In [10]:
for col in df.select_dtypes(include = 'O'):
    print(f'{col} unique values num: {df[col].nunique()}')
    print('-'*50)

industry unique values num: 113
--------------------------------------------------
sector unique values num: 10
--------------------------------------------------
recommendationKey unique values num: 6
--------------------------------------------------
financialCurrency unique values num: 2
--------------------------------------------------


Looks like it is best have to label encode the `industry` with 113 unique categories.
`sector` and `recommendationKey` can safely be one-hot-encoded as it will expand the dataset by only 16 columns.
And we will just binarize the `financialCurrency` column

In [11]:
def binarize(df, col):
    uniques = list(df[col].dropna().unique())
    df[col] = [1 if val == uniques[0] else 0 for val in df[col]]
    return df

In [12]:
def one_hot_encode(df, col):
    dummies = pd.get_dummies(df[col], prefix = col).astype(int)
    df.drop(col, axis = 1, inplace = True)
    df = pd.concat([df, dummies], axis = 1)
    return df

In [13]:
def factorize(df, col):
    mapping = {name: ix for ix, name in enumerate(df[col].unique())}
    df[col] = df[col].map(mapping)
    return df

In [14]:
df = binarize(df, 'financialCurrency')
print(df['financialCurrency'].head())

0    1
1    1
2    0
3    1
4    1
Name: financialCurrency, dtype: int64


In [15]:
print(df.shape)
df = one_hot_encode(df, 'sector')
df = one_hot_encode(df, 'recommendationKey')
print(df.shape)

(2030, 213)
(2030, 227)


In [16]:
df = factorize(df, 'industry')
print(df.dtypes.value_counts())

float64    207
int64       19
bool         1
dtype: int64


In [17]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Deal with missing values

In [18]:
df.isnull().sum()

Id                                  0
industry                            0
fullTimeEmployees                 102
auditRisk                         625
boardRisk                         625
                                 ... 
recommendationKey_hold              0
recommendationKey_none              0
recommendationKey_sell              0
recommendationKey_strong_buy        0
recommendationKey_underperform      0
Length: 227, dtype: int64

In [19]:
print('Missing data stats:\n')
under_10_percent = 0
ten_to_20_percent = 0
twenty_to_50_percent = 0
over_50_percent = 0
for col in df:
    if np.any(df[col].isnull()):
        percent_missing = np.round(100*df[col].isnull().sum()/len(df),2)
        if percent_missing < 10:
            under_10_percent+=1
        elif 10 <= percent_missing < 20:
            ten_to_20_percent+=1
        elif 20 <= percent_missing < 50:
            twenty_to_50_percent+=1
        else:
            over_50_percent+=1

print(f'ncols with under 10% missing data:\t{under_10_percent}')
print(f'ncols with 10-20% missing data:\t\t{ten_to_20_percent}')
print(f'ncols with 20-50% missing data:\t\t{twenty_to_50_percent}')
print(f'ncols with over 50% missing data:\t{over_50_percent}')

Missing data stats:

ncols with under 10% missing data:	134
ncols with 10-20% missing data:		44
ncols with 20-50% missing data:		16
ncols with over 50% missing data:	1


Many columns have missing data, but the majority of cols with NaNs have under 10% missing values. It's safe to fill all missing values by median.

In [20]:
# check if there are any inf values in data
df[df == np.inf].count().sum()

17

In [21]:
# check for missing values in the target columns
train[targets_names].isnull().sum()

Q0_TOTAL_ASSETS                 0
Q0_TOTAL_LIABILITIES            0
Q0_TOTAL_STOCKHOLDERS_EQUITY    0
Q0_GROSS_PROFIT                 0
Q0_COST_OF_REVENUES             0
Q0_REVENUES                     0
Q0_OPERATING_INCOME             0
Q0_OPERATING_EXPENSES           0
Q0_EBITDA                       0
dtype: int64

In [22]:
def impute_missing_by_median(df):
    print(f'Missing values before imputation: {sum(df.isnull().sum())}')
    # first replace inf and -inf with nan
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # then impute nan by median
    for col in df:
        if np.any(df[col].isnull()):
            df[col].fillna(df[col].median(), inplace = True)
    print(f'Missing values after imputation: {sum(df.isnull().sum())}')
    return df

In [23]:
df = impute_missing_by_median(df)

Missing values before imputation: 28653
Missing values after imputation: 0


## Extract train & test without target cols and separate tartgets df with 9 target cols

In [24]:
train = df[df['is_train'] == True]
test = df[df['is_train'] == False]
train.drop('is_train', axis = 1, inplace=True)
test.drop('is_train', axis = 1, inplace=True)
test.reset_index(drop = True, inplace = True)

test.drop(targets_names, axis = 1, inplace = True)
targets = train[targets_names].copy()
train.drop(targets_names, axis = 1, inplace = True)

<ipython-input-24-996949eefa5f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop('is_train', axis = 1, inplace=True)
<ipython-input-24-996949eefa5f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop('is_train', axis = 1, inplace=True)
<ipython-input-24-996949eefa5f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(targets_names, axis = 1, inplace = True)
<ipython-input-24-996949eefa5f>:9: SettingWithCopyWa

## Validate XGB model on train data
### Since we have 9 independent target columns, I will train 9 separate separate models on train features to predict each target.

In [25]:
targets

,Q0_TOTAL_ASSETS,Q0_TOTAL_LIABILITIES,Q0_TOTAL_STOCKHOLDERS_EQUITY,Q0_GROSS_PROFIT,Q0_COST_OF_REVENUES,Q0_REVENUES,Q0_OPERATING_INCOME,Q0_OPERATING_EXPENSES,Q0_EBITDA
0,1.192950e+09,1.055816e+09,1.371340e+08,25598000.0,6.189900e+07,4.299200e+07,1.522200e+07,10383000.0,15215000.0
1,1.219954e+09,8.350460e+08,3.849080e+08,111786000.0,2.177960e+08,3.295820e+08,2.890800e+07,85522000.0,26264000.0
2,-1.358200e+11,-3.797200e+10,-9.784800e+10,-435000000.0,4.445000e+09,4.010000e+09,-1.472000e+09,219000000.0,-654000000.0
3,8.329800e+07,1.604000e+07,-3.163300e+07,494000.0,3.339000e+06,3.833000e+06,-1.234800e+07,12842000.0,-12348000.0
4,1.485460e+08,5.655500e+07,9.199100e+07,1761000.0,-2.375000e+06,-6.140000e+05,-1.009800e+07,11859000.0,-10098000.0
...,...,...,...,...,...,...,...,...,...
1619,3.539600e+08,1.997590e+08,1.542010e+08,48741000.0,4.569300e+07,9.443400e+07,2.616000e+06,46125000.0,2616000.0
1620,1.061530e+08,3.949300e+07,6.811700e+07,1077000.0,1.690000e+05,1.246000e+06,-6.705000e+06,7782000.0,-6705000.0
1621,5.203150e+08,3.807960e+08,3.233080e+08,27808000.0,1.123840e+08,1.401920e+08,7.061000e+06,9417000.0,18391000.0
1622,1.997210e+08,2.692100e+07,1.728000e+08,0.0,0.000000e+00,0.000000e+00,-3.391900e+07,33919000.0,-33919000.0


In [26]:
model = XGBRegressor()
from sklearn.svm import SVC,SVR

from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
REG_svm = SVR(kernel='rbf',C= 1.0, gamma='auto')

rfc=RandomForestClassifier()
dfc=DecisionTreeClassifier()
d_tree=DecisionTreeRegressor(criterion='squared_error',splitter='random',max_depth=4)
rf_forest=RandomForestRegressor(n_estimators=100,criterion='squared_error',max_features='sqrt',max_depth=4)
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
#rfc_param={"n_estimators":range(5,150,10),"max_features":['sqrt'] ,'max_depth':range(2,50,1) ,"min_samples_leaf":range(2,15,1) ,"min_samples_split":range(2,25,1)  }
#model = XGBRegressor()
#reg_tree=RandomizedSearchCV(estimator= d_tree,scoring='r2', param_distributions= rfc_param,cv=2,n_jobs=-1,random_state=42)

In [27]:
print('Cross validation R2 scores for each target:\n')
cross_val_score_results = {}
for target in targets:
    score = np.round(np.mean(cvs(rf_forest, train, targets[target], cv=10, scoring='r2')),2)
    cross_val_score_results[target] = score
    print(f'{target} -> {score}')
print(f'\nMean R2 score across all targets: {np.mean(list(cross_val_score_results.values()))}')

Cross validation R2 scores for each target:

Q0_TOTAL_ASSETS -> 0.83
Q0_TOTAL_LIABILITIES -> 0.7
Q0_TOTAL_STOCKHOLDERS_EQUITY -> 0.77
Q0_GROSS_PROFIT -> 0.84
Q0_COST_OF_REVENUES -> 0.87
Q0_REVENUES -> 0.91
Q0_OPERATING_INCOME -> 0.68
Q0_OPERATING_EXPENSES -> 0.78
Q0_EBITDA -> 0.41

Mean R2 score across all targets: 0.7544444444444445


Arbitrary results. EBITDA R2 is negative, but the rest are not bad. Let's try to upload a first submission.

## Train same model on the whole trains set, predict on test, save to dictionary

In [28]:
preds = {}
for target in targets:
    rf_forest.fit(train, targets[target])
    pred = rf_forest.predict(test)
    preds[target] = pred
    print(f'Finished train/predict for: {target}')

Finished train/predict for: Q0_TOTAL_ASSETS
Finished train/predict for: Q0_TOTAL_LIABILITIES
Finished train/predict for: Q0_TOTAL_STOCKHOLDERS_EQUITY
Finished train/predict for: Q0_GROSS_PROFIT
Finished train/predict for: Q0_COST_OF_REVENUES
Finished train/predict for: Q0_REVENUES
Finished train/predict for: Q0_OPERATING_INCOME
Finished train/predict for: Q0_OPERATING_EXPENSES
Finished train/predict for: Q0_EBITDA


## Make submission

In [29]:
sub = pd.read_csv('/kaggle/input/financial-performance-prediction/sample_submission.csv')

In [30]:
# sanity check
sub.Id == test.Id

0      True
1      True
2      True
3      True
4      True
       ... 
401    True
402    True
403    True
404    True
405    True
Name: Id, Length: 406, dtype: bool

In [31]:
for target in preds:
    sub[target] = preds[target]
sub.head()

,Id,Q0_REVENUES,Q0_COST_OF_REVENUES,Q0_GROSS_PROFIT,Q0_OPERATING_EXPENSES,Q0_EBITDA,Q0_OPERATING_INCOME,Q0_TOTAL_ASSETS,Q0_TOTAL_LIABILITIES,Q0_TOTAL_STOCKHOLDERS_EQUITY
0,480,8.241275e+07,7.165119e+07,4.077679e+07,5.617852e+07,1.815882e+07,9.656212e+06,1.612206e+09,1.002670e+09,5.247569e+08
1,573,7.349296e+08,2.571748e+08,2.351013e+08,1.688654e+08,1.815882e+07,1.233766e+07,1.731696e+09,1.002670e+09,6.480068e+08
2,1967,8.241275e+07,7.165119e+07,4.077679e+07,5.617852e+07,1.815882e+07,9.656212e+06,1.612206e+09,1.002670e+09,5.247569e+08
3,57,5.499801e+08,1.826125e+08,1.927335e+08,1.485328e+08,1.815882e+07,1.685489e+07,2.589125e+09,1.524834e+09,1.619934e+09
4,131,9.934412e+08,6.192011e+08,2.464352e+08,1.682185e+08,1.815882e+07,2.497256e+07,2.160349e+09,1.399015e+09,7.035041e+08


In [32]:
sub.to_csv('submission.csv', index = False)

In [33]:
#ids = np.array(sub.Id)
#preds.keys()

In [34]:
# Write the submission file
#np.savetxt(
#    'submission.csv',
#    np.rec.fromarrays([ids] + [pred for pred in list(preds.values())]),
#    fmt=['%s', '%d', '%d', '%d', '%d', '%d', '%d', '%d', '%d', '%d'],
#    delimiter=',',
#    header='Id,Q0_TOTAL_ASSETS,Q0_TOTAL_LIABILITIES,Q0_TOTAL_STOCKHOLDERS_EQUITY,Q0_GROSS_PROFIT,Q0_COST_OF_REVENUES,Q0_REVENUES,Q0_OPERATING_INCOME,Q0_OPERATING_EXPENSES,Q0_EBITDA',
#    comments='submission description from code',
#)